# 1. RuneScape Item Price Exploration

This notebook is for exploring the raw price data collected from the OSRS Wiki API. The goal is to visualize the data, understand its properties, and get ideas for feature engineering.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 7)

## Load Data

Let's load the data for a single item. We'll use the Twisted Bow as an example. The data is in JSON format, so we'll need to parse it and convert the timestamp.

In [ ]:
RAW_DATA_PATH = Path('../data/raw')
ITEM_FILE = RAW_DATA_PATH / 'twisted_bow_1h.json'

if not ITEM_FILE.exists():
    print(f"File not found: {ITEM_FILE}")
    print("Please run `python src/main.py collect` first.")
else:
    # The API data is nested under a 'data' key
    df_raw = pd.read_json(ITEM_FILE)
    df = pd.json_normalize(df_raw['data'])
    
    # Convert unix timestamp to datetime and set as index
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df = df.set_index('timestamp')
    
    print(df.info())
    display(df.head())

## Visualize Price Over Time

A simple line plot is the best way to get a first look at the time series.

In [ ]:
if 'df' in locals():
    # Let's create an average price column for simplicity
    df['avgPrice'] = (df['avgHighPrice'] + df['avgLowPrice']) / 2

    df['avgPrice'].plot(title='Twisted Bow Price Over Time')
    plt.ylabel('Price (GP)')
    plt.show()

## Next Steps

From here, you can:
- Plot rolling averages to see trends.
- Analyze price distribution with a histogram.
- Check for seasonality (e.g., weekly or daily patterns).